In [ ]:
import os
import sys
from time import sleep

import GCode
import GRBL
import numpy as np

# Laser Material Calibration

Loop over laser power settings and feed rate to determine how a given material looks.

# Code:

In [2]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()
cnc.laser_mode = 1


ok


In [3]:
print(cnc.laser_mode)


1.0


In [4]:
cnc.home()


In [5]:
cnc.status


'<Idle|MPos:-259.000,-174.000,-1.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [6]:
cnc.cmd("G28")


['ok', 'ok']

In [20]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=200)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    return program


In [23]:
def line(X=0, Y0=0, Yf=20, feed=100, power=255, dynamic_power=True):
    prog = GCode.GCode()
    prog.G0(X=X, Y=Y0)
    if dynamic_power:
        prog.M4(S=power)
    else:
        prog.M3(S=power)
    prog.G1(X=X, Y=Yf, F=feed)
    prog.M5()
    return prog


In [24]:
dX = 0.1
Xf = 2
dXs = np.arange(0, Xf + dX, dX)
dXs


array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ])

In [25]:
np.cumsum(dXs)


array([ 0. ,  0.1,  0.3,  0.6,  1. ,  1.5,  2.1,  2.8,  3.6,  4.5,  5.5,
        6.6,  7.8,  9.1, 10.5, 12. , 13.6, 15.3, 17.1, 19. , 21. ])

In [26]:
line_len = 15
Yf = 10
cnc.run(init())
for feed in [50, 100, 150, 200, 300]:
    X0 = 10
    Y0 = Yf
    Yf = Y0 + line_len
    for power in [30, 40, 50, 75, 100, 125]:
        for X in np.cumsum(dXs) + X0:
            L = line(X=X, Y0=Y0, Yf=Yf, feed=feed, power=power)
            cnc.run(L)
        X0 = X + 5


In [27]:
cnc.cmd("G28")


['ok', 'ok']